In [4]:
import pandas as pd
import numpy as np

import optuna

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adagrad, Adadelta, Nadam

from sklearn.metrics import roc_auc_score

In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:
print(tf.__version__)

2.20.0


In [76]:

data = pd.read_csv(r'C:\Users\Aysel Quliyeva\Desktop\data science with python\lesson 25\Employee.csv')
data


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1
...,...,...,...,...,...,...,...,...,...
4648,Bachelors,2013,Bangalore,3,26,Female,No,4,0
4649,Masters,2013,Pune,2,37,Male,No,2,1
4650,Masters,2018,New Delhi,3,27,Male,No,5,1
4651,Bachelors,2012,Bangalore,3,30,Male,Yes,2,0


In [77]:
data.describe(include='all')

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
count,4653,4653.000000,4653,4653.000000,4653.000000,4653,4653,4653.000000,4653.000000
unique,3,NaN,3,NaN,NaN,2,2,NaN,NaN
top,Bachelors,NaN,Bangalore,NaN,NaN,Male,No,NaN,NaN
freq,3601,NaN,2228,NaN,NaN,2778,4175,NaN,NaN
mean,NaN,2015.062970,NaN,2.698259,29.393295,NaN,NaN,2.905652,0.343864
std,NaN,1.863377,NaN,0.561435,4.826087,NaN,NaN,1.558240,0.475047
min,NaN,2012.000000,NaN,1.000000,22.000000,NaN,NaN,0.000000,0.000000
25%,NaN,2013.000000,NaN,3.000000,26.000000,NaN,NaN,2.000000,0.000000
50%,NaN,2015.000000,NaN,3.000000,28.000000,NaN,NaN,3.000000,0.000000
75%,NaN,2017.000000,NaN,3.000000,32.000000,NaN,NaN,4.000000,1.000000


In [78]:
data.isnull().sum()

Education                    0
JoiningYear                  0
City                         0
PaymentTier                  0
Age                          0
Gender                       0
EverBenched                  0
ExperienceInCurrentDomain    0
LeaveOrNot                   0
dtype: int64

In [79]:
data = pd.get_dummies(data, drop_first=True, dtype=int)

data

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,LeaveOrNot,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes
0,2017,3,34,0,0,0,0,0,0,1,0
1,2013,1,28,3,1,0,0,0,1,0,0
2,2014,3,38,2,0,0,0,1,0,0,0
3,2016,3,27,5,1,1,0,0,0,1,0
4,2017,3,24,2,1,1,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
4648,2013,3,26,4,0,0,0,0,0,0,0
4649,2013,2,37,2,1,1,0,0,1,1,0
4650,2018,3,27,5,1,1,0,1,0,1,0
4651,2012,3,30,2,0,0,0,0,0,1,1


In [80]:
targets = data['LeaveOrNot']

inputs = data.drop(['LeaveOrNot'],axis=1)

In [81]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(inputs)

scaled = scaler.transform(inputs)

inputs_scaled = pd.DataFrame(scaled, columns=inputs.columns)

inputs_scaled

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes
0,1.039638,0.537503,0.954645,-1.864901,-0.480575,-0.200022,-0.575282,-0.612041,0.821551,-0.338365
1,-1.107233,-3.025177,-0.288732,0.060554,-0.480575,-0.200022,-0.575282,1.633878,-1.217210,-0.338365
2,-0.570515,0.537503,1.783563,-0.581264,-0.480575,-0.200022,1.738277,-0.612041,-1.217210,-0.338365
3,0.502921,0.537503,-0.495961,1.344191,2.080840,-0.200022,-0.575282,-0.612041,0.821551,-0.338365
4,1.039638,0.537503,-1.117650,-0.581264,2.080840,-0.200022,-0.575282,1.633878,0.821551,2.955387
...,...,...,...,...,...,...,...,...,...,...
4648,-1.107233,0.537503,-0.703191,0.702373,-0.480575,-0.200022,-0.575282,-0.612041,-1.217210,-0.338365
4649,-1.107233,-1.243837,1.576334,-0.581264,2.080840,-0.200022,-0.575282,1.633878,0.821551,-0.338365
4650,1.576356,0.537503,-0.495961,1.344191,2.080840,-0.200022,1.738277,-0.612041,0.821551,-0.338365
4651,-1.643951,0.537503,0.125727,-0.581264,-0.480575,-0.200022,-0.575282,-0.612041,0.821551,2.955387


In [82]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size=0.2, random_state=42)

In [83]:
def create_model(trial):
    # Building artificial neural network
    model = Sequential()

     # we add 2 hidden layers and 1 output layer
    model.add(Dense(units=trial.suggest_int('units_layer1', 6, 32), activation='relu'))
    model.add(Dense(units=trial.suggest_int('units_layer2', 6, 32), activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))

    # Suggest hyperparameters for the optimizer
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop', 'adagrad'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    elif optimizer_name == 'adagrad':
        optimizer = Adagrad(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])
    
    return model

In [84]:
def optimal(trial):
    
    # Suggest the number of epochs and batch size
    epochs = trial.suggest_int('epochs', 10, 50)
    batch_size = trial.suggest_int('batch_size', 16, 64)
    
    model = create_model(trial)
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred)
    
    return auc

study = optuna.create_study(direction='maximize')
study.optimize(optimal, n_trials=10)

print(f"Best trial: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2025-09-27 12:55:51,599] A new study created in memory with name: no-name-036fa74c-24f6-4206-b27c-e577332e99f8


Epoch 1/11


C:\Users\Aysel Quliyeva\AppData\Local\Temp\ipykernel_22556\3644950783.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.4198 - loss: 0.6805  
Epoch 2/11
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.4302 - loss: 0.6753
Epoch 3/11
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.4400 - loss: 0.6708
Epoch 4/11
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.4491 - loss: 0.6670
Epoch 5/11
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.4588 - loss: 0.6633
Epoch 6/11
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.4675 - loss: 0.6600
Epoch 7/11
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.4765 - loss: 0.6571
Epoch 8/11
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.4858 - loss: 0.6542  
Epoch 9/11
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.4945 - loss: 0.6516
Epoch 10/11
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.5040 - loss: 0.6491
Epoch 11/11
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.5123 - loss: 0.6468
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


[I 2025-09-27 12:55:54,418] Trial 0 finished with value: 0.5070272202645423 and parameters: {'epochs': 11, 'batch_size': 59, 'units_layer1': 28, 'units_layer2': 29, 'optimizer': 'adagrad', 'learning_rate': 0.000542649127459985}. Best is trial 0 with value: 0.5070272202645423.


...

Epoch 1/43


C:\Users\Aysel Quliyeva\AppData\Local\Temp\ipykernel_22556\3644950783.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


138/138 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.5360 - loss: 0.6640
Epoch 2/43
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.5572 - loss: 0.6535
Epoch 3/43
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.5744 - loss: 0.6474
Epoch 4/43
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.5872 - loss: 0.6427
Epoch 5/43
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6016 - loss: 0.6386
Epoch 6/43
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6132 - loss: 0.6349
Epoch 7/43
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6254 - loss: 0.6314
Epoch 8/43
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6371 - loss: 0.6281
Epoch 9/43
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6475 - loss: 0.6249
Epoch 10/43
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6568 - loss: 0.6217
Epoch 11/43
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6656 - loss: 0.6187
Epoch 12/43
138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.6743 - loss: 0.6157
Epoch 13/43
138/138 ━━━━━━━━━━━━━━━━

[I 2025-09-27 12:56:59,006] Trial 9 finished with value: 0.7670956539502579 and parameters: {'epochs': 43, 'batch_size': 27, 'units_layer1': 8, 'units_layer2': 21, 'optimizer': 'adagrad', 'learning_rate': 0.0033513934149419504}. Best is trial 6 with value: 0.8985087584903734.


Best trial: 0.8985087584903734
Best hyperparameters: {'epochs': 24, 'batch_size': 24, 'units_layer1': 32, 'units_layer2': 6, 'optimizer': 'rmsprop', 'learning_rate': 0.001641019487079872}


In [85]:
best_params = study.best_trial.params

best_params

{'epochs': 24,
 'batch_size': 24,
 'units_layer1': 32,
 'units_layer2': 6,
 'optimizer': 'rmsprop',
 'learning_rate': 0.001641019487079872}

In [86]:
best_model = Sequential()
best_model.add(Dense(units=32, activation='relu'))
best_model.add(Dense(units=6, activation='relu'))
best_model.add(Dense(1, activation='sigmoid'))

In [87]:
if best_params['optimizer'] == 'adam':
    best_optimizer = Adam(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'sgd':
    best_optimizer = SGD(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'rmsprop':
    best_optimizer = RMSprop(learning_rate=best_params['learning_rate'])
elif best_params['optimizer'] == 'adagrad':
    best_optimizer = Adagrad(learning_rate=best_params['learning_rate'])


In [88]:
best_model.compile(optimizer=best_optimizer, loss='binary_crossentropy', metrics=['AUC'])

In [89]:
def evaluate(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train, epochs=24, batch_size=24)
    
    '''Predictions and probabilities for the training set'''
    
    y_train_prob = model.predict(X_train)

    '''Predictions and probabilities for the test set'''
    
    y_test_prob = model.predict(X_test)

    '''Calculate metrics for the training set''' 
    
    roc_train_prob = roc_auc_score(y_train, y_train_prob)
    gini_train_prob = roc_train_prob * 2 - 1
    

    '''Calculate metrics for the test set'''
    
    roc_test_prob = roc_auc_score(y_test, y_test_prob)
    gini_test_prob = roc_test_prob * 2 - 1
    

    results = pd.DataFrame({
        'Dataset': ['Train', 'Test'],
        'Gini': [gini_train_prob * 100, gini_test_prob * 100],
    
    })

    return results

In [90]:
evaluate(best_model, X_train, y_train, X_test, y_test)

Epoch 1/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - AUC: 0.6988 - loss: 0.5965
Epoch 2/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7814 - loss: 0.5437
Epoch 3/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7972 - loss: 0.5134
Epoch 4/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8101 - loss: 0.4914
Epoch 5/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8204 - loss: 0.4738
Epoch 6/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8313 - loss: 0.4595
Epoch 7/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8375 - loss: 0.4491
Epoch 8/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8417 - loss: 0.4420
Epoch 9/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8458 - loss: 0.4349
Epoch 10/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8487 - loss: 0.4295
Epoch 11/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8521 - loss: 0.4243
Epoch 12/24
156/156 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.8519 - loss: 0.4203
Epoch 13/24
156/156 ━━━━━

,Dataset,Gini
0,Train,76.937730
1,Test,78.271283


# deploy

In [101]:
deploy = {
    'Education': ['Bachelors', 'PHD', 'Bachelors', 'PHD', 'Masters'],
    'JoiningYear': [2017, 2013, 2014, 2016, 2017],
    'City': ['Bangalore', 'Pune', 'New Delhi', 'Bangalore', 'Pune'],
    'PaymentTier': [3, 1, 3, 3, 3],
    'Age': [40, 32, 38, 21, 24],
    'Gender': ['Male', 'Female', 'Female', 'Male', 'Male'],
    'EverBenched': ['No', 'No', 'No', 'No', 'Yes'],
    'ExperienceInCurrentDomain': [0, 3, 2, 5, 2],
}

df = pd.DataFrame(deploy)

df

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
0,Bachelors,2017,Bangalore,3,40,Male,No,0
1,PHD,2013,Pune,1,32,Female,No,3
2,Bachelors,2014,New Delhi,3,38,Female,No,2
3,PHD,2016,Bangalore,3,21,Male,No,5
4,Masters,2017,Pune,3,24,Male,Yes,2


In [102]:
X_train.columns

Index(['JoiningYear', 'PaymentTier', 'Age', 'ExperienceInCurrentDomain',
       'Education_Masters', 'Education_PHD', 'City_New Delhi', 'City_Pune',
       'Gender_Male', 'EverBenched_Yes'],
      dtype='object')

In [103]:
test=df.copy()

test

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain
0,Bachelors,2017,Bangalore,3,40,Male,No,0
1,PHD,2013,Pune,1,32,Female,No,3
2,Bachelors,2014,New Delhi,3,38,Female,No,2
3,PHD,2016,Bangalore,3,21,Male,No,5
4,Masters,2017,Pune,3,24,Male,Yes,2


In [104]:
test = pd.get_dummies(test, dtype=int)

test

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Bachelors,Education_Masters,Education_PHD,City_Bangalore,City_New Delhi,City_Pune,Gender_Female,Gender_Male,EverBenched_No,EverBenched_Yes
0,2017,3,40,0,1,0,0,1,0,0,0,1,1,0
1,2013,1,32,3,0,0,1,0,0,1,1,0,1,0
2,2014,3,38,2,1,0,0,0,1,0,1,0,1,0
3,2016,3,21,5,0,0,1,1,0,0,0,1,1,0
4,2017,3,24,2,0,1,0,0,0,1,0,1,0,1


In [105]:
test = test[['JoiningYear', 'PaymentTier', 'Age', 'ExperienceInCurrentDomain',
       'Education_Masters', 'Education_PHD', 'City_New Delhi', 'City_Pune',
       'Gender_Male', 'EverBenched_Yes']]

In [106]:
scaler.fit(test)

scaled = scaler.transform(test)

df_test_scaled = pd.DataFrame(scaled, columns=test.columns)

df_test_scaled

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Masters,Education_PHD,City_New Delhi,City_Pune,Gender_Male,EverBenched_Yes
0,0.984732,0.5,1.202676,-1.477098,-0.5,-0.816497,-0.5,-0.816497,0.816497,-0.5
1,-1.477098,-2.0,0.133631,0.369274,-0.5,1.224745,-0.5,1.224745,-1.224745,-0.5
2,-0.861640,0.5,0.935414,-0.246183,-0.5,-0.816497,2.0,-0.816497,-1.224745,-0.5
3,0.369274,0.5,-1.336306,1.600189,-0.5,1.224745,-0.5,-0.816497,0.816497,-0.5
4,0.984732,0.5,-0.935414,-0.246183,2.0,-0.816497,-0.5,1.224745,0.816497,2.0


In [107]:
df['Churn_employee'] = best_model.predict(df_test_scaled)


df

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,Churn_employee
0,Bachelors,2017,Bangalore,3,40,Male,No,0,0.257002
1,PHD,2013,Pune,1,32,Female,No,3,0.948485
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0.127933
3,PHD,2016,Bangalore,3,21,Male,No,5,0.057703
4,Masters,2017,Pune,3,24,Male,Yes,2,0.881822
